### 導入套件

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from linearmodels import FamaMacBeth
from stargazer.stargazer import Stargazer, LineLocation


### 整理資料

In [2]:
df = pd.read_csv("C:/Users/USER/Desktop/MLY_RA/data/OHLCV_20130101_20230331_v1_1to24180WITHinactive.csv") #讀csv檔案進來
df2 = df[['close', 'volume_24h', 'market_cap', 'id', 'name', 'symbol', 'date']] #選出想要的columns
df3 = df2.dropna(subset=['date']) #把date欄位 資料是na的刪掉
df3['date'] = pd.to_datetime(df3['date']) #把date欄位的資料轉成datetime 形式

In [3]:
#把每個id的date中有重複的刪掉，保留一個
df4 = df3.groupby('id')[['close', 'volume_24h','market_cap','name', 'symbol', 'date']].apply(lambda x : x.drop_duplicates(subset=['date']))
df5 = df4.reset_index()
df5 = df5.set_index('date')
#因為資料有些斷點，用resample把日資料補齊
df6 = df5.groupby('id')[['close', 'volume_24h','market_cap','name','symbol']].apply(lambda x: x.resample('D').asfreq())

### 定義函式

In [4]:
#定義函式計算每日報酬率，投組在資訊區間的第一天建構，由於投組資訊會落後一期，因此以weekly return舉例來說:(1/8的收盤價/1/1 的收盤價 -1)為1/1號所建構投組的週報酬率
def ret(df,n): #定義一個計算log return的function
    raw_data = df.copy()
    raw_data = raw_data.assign(ret=np.log(raw_data.close).groupby(raw_data.id).diff(periods = n))
    return raw_data

def ret_for_prior_week(df,n): #定義一個計算week_log return的function
    raw_data = df.copy()
    raw_data = raw_data.assign(week_ret=np.log(raw_data.close).groupby(raw_data.id).diff(periods = n))
    return raw_data


def ret2(a,n1,n2):
    x = np.log(a.shift(n1)/a.shift(n2))
    return x

In [5]:
from scipy.stats.mstats import winsorize
def winsorwize_nonret(x,df_list): #winsorwize non return var。縮尾處理時不考慮缺值，避免縮尾後出現缺值
    df = x.copy()
    df = df[df['market_cap']> 2000000] #排除市值低於200萬
    df = df[df['volume_24h'] > 0]
    df.replace(np.inf, np.nan, inplace=True)#將inf替換成NA
    for i in df_list:
        mask = df[i].notna()
        df.loc[mask,i] = winsorize(df[i].loc[mask],limits = [0.01,0.01])
    
    return df

def winsorwize_nonret2(df,df_list,group_list):
    for i in df_list:
        df[i] = df.groupby(group_list)[i].transform(lambda x: winsorize(x,limits=[0.01,0.01],nan_policy='omit'))
    return df

def winsorwize(x): #winsorwize ret
    df = x.copy()
    df = df[df['market_cap']> 2000000] #排除市值低於200萬
    df = df[df['volume_24h'] > 0]
    df.replace(np.inf, np.nan, inplace=True)
    return df

In [6]:
#清除index(以防分組處理時報錯)，原因:會有index不唯一的狀況，解決辦法:將id及date皆設為index(id和date要各複製一個非同名的新欄位作為index)
df7 = df6.reset_index()

### 將資料導入R中做周資料的轉換

In [7]:
#將資料依年份及id排序
#data = df7.copy().sort_values(by = ['date','id'])
# 将字符串日期转换为 datetime 对象
#data['date'] = pd.to_datetime(data['date'])
#outputpath = 'transform_week.csv'
#data.to_csv(outputpath,sep=',',index = False,header=True)

#將資料轉到R中做周次和年份的處理(因python是以isocalendor作為基準，每年的第一周是從第一個禮拜一開始計算)

### 將轉換好的資料導回python

In [8]:
data = pd.read_csv("C:/Users/USER/Desktop/MLY_RA/data/transform_finished.csv")
data['time'] = pd.to_datetime(data['date'])
data['id_name'] = data['id']

data = data.reset_index(drop=True)
data = data.set_index(['id_name','time'],drop = True)#此處index中的id_name為level0，time為level1

#data = data.sort_values('date')
#獲取年份添加到dataframe
#data['year'] = data['date'].dt.year

# 获取周数并添加到 DataFrame
#data['week'] = data['date'].dt.strftime("%U", d)
#data['week'] = data['week'].apply(lambda x:52 if x == 53 else x)



### 計算TABLE1所需變數

In [9]:

#TABLE1所需因子的計算
data = ret(data,1)
data['VaR_5'] = data.groupby('id')['ret'].rolling(90,min_periods = 90).quantile(0.05).reset_index(0,drop=True)*(-1)
#reset_index(0,drop = True)中的0代表index的level，指的是索引的層級，若指定兩個變數作為索引，那index就有0跟1的層級(set_index時誰放前面誰的level就越前面)，以此類推。rolling函數會回傳一個series or dataframe，其中如有利用到groupby函數，則groupby的對象會做為新series or dataframe的索引，並以level0的層級加進index，以此為例:id為level0，id_name為level1，time為level2。若是回傳series，並且想直接作為某dataframe的新一列，則index必須對齊，因此必須捨棄掉level0(若有兩個groupby的對象就是0和1，以此類推)
data['Size'] = np.log(data['market_cap'])
data['Mom'] = ret2(data.groupby('id')['close'],7,14)
#Vhigh、Vlow
data['condition'] = data.groupby('id')['volume_24h'].rolling(49,min_periods = 49).quantile(0.1).reset_index(0,drop=True)
data['condition_2'] = data.groupby('id')['volume_24h'].rolling(49,min_periods = 49).quantile(0.9).reset_index(0,drop=True)
data['volume_last_day'] = data.groupby(['id','year','week'])['volume_24h'].transform('last')
data['Vhigh'] = np.where(data['volume_last_day'] > data['condition_2'], 1, 0)
data['Vlow'] = np.where(data['volume_last_day'] < data['condition'], 1, 0)
###
data['Vol'] = data.groupby('id')['ret'].rolling(90,min_periods = 90).std().reset_index(0,drop=True)
data['Prc'] = np.log(data.groupby(['id','year','week'])['close'].transform('last'))
data['Maxdprc'] = np.log(data.groupby(['id','year','week'])['close'].transform('max'))
#Abvol、Illiq、Prcvol、Stdprcvol
data['dollar_volume'] = data['volume_24h']*data['close']
data['dollar_volume_mean'] = data.groupby(['id','year','week'])['dollar_volume'].transform('mean')
data['log_dollar_volume'] = np.log(data['dollar_volume'])
data['avg_log_dv_12_weeks'] = data.groupby('id')['log_dollar_volume'].rolling(84,min_periods = 84).mean().reset_index(0,drop=True)
data['avg_log_dv_12_weeks_all'] = data.groupby('date')['avg_log_dv_12_weeks'].transform('mean')
data['Abvol'] = data['avg_log_dv_12_weeks']-data['avg_log_dv_12_weeks_all']
data['Illiq'] = np.abs(data['ret'])/data['dollar_volume_mean']
data['Prcvol'] = np.log(data.groupby(['id','year','week'])['dollar_volume'].transform('mean'))
data['Stdprcvol'] = np.log(data.groupby(['id','year','week'])['dollar_volume'].transform('std'))
###
data['Max'] = data.groupby('id')['ret'].rolling(28).apply(lambda x: np.mean(sorted(x,reverse=True)[:5])).reset_index(0,drop=True)


d:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
d:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
d:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
d:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
d:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
d:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in lo

In [10]:
#建構market_return
data['MKT_value'] = data.groupby('date')['market_cap'].transform('sum')
data['weight'] = data['market_cap']/data['MKT_value']
data['wret'] = data['ret']*data['weight']
data['MKT_ret'] = data.groupby('date')['wret'].transform('sum')
data = data.reset_index()#只能做一次，如果重複執行出現錯誤就把這段反白
data = data.set_index('time')#將index設為日期(方便等等與Rf直接以index合併)



導入一個月期公債

In [11]:
#引入一個月期公債利率
rf = pd.read_csv("C:/Users/USER/Desktop/MLY_RA/data/DGS1MO.csv")
rf['time'] = pd.to_datetime(rf['DATE'])
rf['rf'] = rf['DGS1MO']
rf = rf.reset_index(drop = True)
rf = rf.set_index('time')
rf1 = rf.apply(lambda x: x.resample('D').asfreq())
rf1['rf'] = rf1['rf'].fillna(method='ffill')#假日會沒資料，用禮拜五的資料去補
rf1['rf'] = rf1['rf']/100

計算超額報酬

In [12]:
#計算market_excess_return和excess_ret
data1 = pd.merge(data,rf1['rf'],how = 'left',left_index=True, right_index=True)
data1['MKT_excess'] = data1['MKT_ret'] - data1['rf']*7/365
data1['exret'] = data1['ret'] - data1['rf']*7/365


Beta

In [13]:
#以過去90天的日貨幣報酬與市場報酬回歸所得到的beta
from statsmodels.regression.rolling import RollingOLS

# Read data & adding "intercept" column
data1 = data1.reset_index()
data1 = data1.set_index(['id_name','time'],drop = True)
data1['intercept'] = 1
data1 = data1.groupby('id').filter(lambda x: len(x) > 90)#刪除日資料小於90天的資料
# Groupby then apply RollingOLS(大致要花6分鐘)
reg = data1.groupby('id')[['ret', 'intercept', 'MKT_ret']].apply(lambda g: RollingOLS(g['ret'], g[['intercept', 'MKT_ret',]], window=90,min_nobs=90,missing='drop').fit().params).reset_index(0,drop=True)


d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:270: RuntimeWarning: invalid value encountered in matmul
  wresid = wy - wx @ params
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:270: RuntimeWarning: invalid value encountered in subtract
  wresid = wy - wx @ params
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:279: RuntimeWarning: invalid value encountered in subtract
  centered_tss = np.sum(weights * (y - mean) ** 2)
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:377: RuntimeWarning: invalid value encountered in add
  xpy += add_x.T @ wy[i - 1 : i]
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:272: RuntimeWarning: divide by zero encountered in log
  llf = -np.log(ssr) * nobs2  # concentrated likelihood
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:272: RuntimeWarning: divide by zero encountered in log
  l

In [14]:
#不用落後一期!(因rollingOLS已經是採用過去90天的資料進行回歸，且python是從0開始為索引起始)
#具體來說，如果我們直接用所有資料來建立線性迴歸模型，則迴歸係數，是關於所有x 與所有y 的函數。然而，我們在時是不知道未來的數據點的！如果使用全部資料進行迴歸則相當於未卜先知，會造成嚴重的過度擬合。
#reset_index不要連續跑兩次(除非中間有set_index)
#利用相同的index，直接以df['A'] = df1['B']的方式合併reg的beta、reg1的Ivol、reg2的coskew(在sq_MKT_excess那欄)
reg['beta'] = reg['MKT_ret']
#reg = reg.reset_index()
#reg['beta'] = reg.groupby(['id_name'])['beta_before'].shift(1)
#reg = reg.set_index(['id_name','time'])
data1['Beta'] = reg['beta']


Ivol

In [15]:
#以過去90天的日貨幣超額報酬與市場超額報酬回歸所得到的殘差標準差(Ivol)
gb = data1.groupby(['id'])
reg1 = gb.apply(lambda g: RollingOLS(g['exret'], g[['intercept', 'MKT_excess']], window=90,min_nobs=90,missing='drop').fit().mse_resid).reset_index(0,drop=True)#取出單一個result(如mse)、t_value，就不能用data1.groupby('id')[['ret', 'intercept', 'MKT_ret']]
data1['Ivol'] = reg1
#data1['Ivol'] = data1.groupby(['id'])['Ivol_before'].shift(1)
#data1 = data1.drop(columns= 'Ivol_before')

d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:270: RuntimeWarning: invalid value encountered in matmul
  wresid = wy - wx @ params
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:270: RuntimeWarning: invalid value encountered in subtract
  wresid = wy - wx @ params
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:279: RuntimeWarning: invalid value encountered in subtract
  centered_tss = np.sum(weights * (y - mean) ** 2)
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:377: RuntimeWarning: invalid value encountered in add
  xpy += add_x.T @ wy[i - 1 : i]
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:372: RuntimeWarning: invalid value encountered in subtract
  xpy -= remove_x.T @ wy[i - w - 1 : i - w]
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:372: RuntimeWarning: invalid value encountered in subtract
  xpy -

Coskew

In [16]:
#以過去90天的日貨幣超額報酬與市場超額報酬回歸(平方)所得到的超額市場報酬平方的係數(coskew)

data1['sq_MKT_excess'] = data1['MKT_excess']**2
reg2 = data1.groupby('id')[['exret', 'intercept', 'MKT_excess','sq_MKT_excess']].apply(lambda g: RollingOLS(g['exret'], g[['intercept', 'MKT_excess','sq_MKT_excess']], window=90,min_nobs=90,missing='drop').fit().params).reset_index(0,drop=True)
reg2['coskew'] = reg2['sq_MKT_excess']
#reg2 = reg2.reset_index()
#reg2['coskew'] = reg2.groupby(['id_name'])['coskew_before'].shift(1)
#reg2 = reg2.set_index(['id_name','time'])
data1['Coskew'] = reg2['coskew']


d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:235: RuntimeWarning: invalid value encountered in matmul
  params = wxpwxi @ wxpwy
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:279: RuntimeWarning: invalid value encountered in subtract
  centered_tss = np.sum(weights * (y - mean) ** 2)
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:377: RuntimeWarning: invalid value encountered in add
  xpy += add_x.T @ wy[i - 1 : i]
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:372: RuntimeWarning: invalid value encountered in subtract
  xpy -= remove_x.T @ wy[i - w - 1 : i - w]
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:372: RuntimeWarning: invalid value encountered in subtract
  xpy -= remove_x.T @ wy[i - w - 1 : i - w]
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\rolling.py:372: RuntimeWarning: invalid value encountered in su

### 以VaR5%將虛擬貨幣分為10個投組

計算各投組的市值及平均加權報酬

In [17]:

d = data1.copy().reset_index()

d = d.set_index('time')
d = ret_for_prior_week(d,7)#以處所計算的week_ret、port_ret、port_e_ret都是前一個禮拜的
d1 = d['2016/01/01':'2020/12/31']
d1 = d1.groupby(['id','year','week']).first().reset_index()
d1 = d1.reset_index(drop = True)
d1['time'] = pd.to_datetime(d1['date'])
d1 = d1.set_index(['id_name','time'],drop = True)
d1 = winsorwize(d1)
d1 = winsorwize_nonret2(d1,['VaR_5','Beta','Size','Mom','Vhigh','Vlow','Abvol','Ivol','Vol','Illiq','Coskew','Max','Prc','Maxdprc','Prcvol','Stdprcvol'],['year','week'])
d1['pVaR_5'] = d1.groupby('id')['VaR_5'].shift(1)
d1 = d1.dropna(subset=['pVaR_5'])
d1['VaR5%_group'] = d1.groupby(['date'])['pVaR_5'].transform( lambda x: pd.qcut(x, q=10, labels=range(1,11)))
port_cap = d1.groupby(['VaR5%_group','date'])['market_cap'].transform('sum')
d1['port_weight'] = d1['market_cap']/port_cap
d1['port_wret'] = d1['week_ret']*d1['port_weight']
d1['port_ret'] = d1.groupby(['VaR5%_group','date'])['port_wret'].transform('sum')#計算投組的市值加權報酬

port_size = d1.groupby(['VaR5%_group','date']).size().reset_index(name='port_size')
d1 = pd.merge(d1,port_size,how = 'left',on = ['VaR5%_group','date'])
d1 = d1.reset_index(drop = True)
d1['time'] = pd.to_datetime(d1['date'])
d1['id_name'] = d1['id']
d1 = d1.set_index(['id_name','time'],drop=True)
d1['port_e_wret'] = d1['week_ret']/d1['port_size']
d1['port_e_ret'] = d1.groupby(['VaR5%_group','date'])['port_e_wret'].transform('sum')#計算投組的等值加權報酬

d1['port_excess_ret'] = d1['port_ret'] - d1['rf']*7/365
d1['port_e_excess_ret'] = d1['port_e_ret'] - d1['rf']*7/365
d1

d:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


id  year  week        date       close    volume_24h  \
id_name time                                                                 
1       2016-01-08     1  2016     2  2016-01-08  453.230011  5.699300e+07   
        2016-01-15     1  2016     3  2016-01-15  364.330994  1.533510e+08   
        2016-01-22     1  2016     4  2016-01-22  382.492004  9.154660e+07   
        2016-01-29     1  2016     5  2016-01-29  379.473999  8.612530e+07   
        2016-02-05     1  2016     6  2016-02-05  386.549011  4.382500e+07   
...                  ...   ...   ...         ...         ...           ...   
7046    2020-12-23  7046  2020    52  2020-12-23    0.612376  2.409312e+07   
7048    2020-12-23  7048  2020    52  2020-12-23   14.189963  4.047784e+06   
7083    2020-12-23  7083  2020    52  2020-12-23    3.313970  8.383838e+08   
7096    2020-12-23  7096  2020    52  2020-12-23    0.010429  2.684505e+06   
7133    2020-12-23  7133  2020    52  2020-12-23   26.164306  4.290594e+05   

                      market_cap                  name  symbol       ret  ...  \
id_name time                                                              ...   
1       2016-01-08  6.825700e+09               Bitcoin     BTC -0.010574  ...   
        2016-01-15  5.496598e+09               Bitcoin     BTC -0.166434  ...   
        2016-01-22  5.780764e+09               Bitcoin     BTC -0.070088  ...   
        2016-01-29  5.745986e+09               Bitcoin     BTC -0.002145  ...   
        2016-02-05  5.865137e+09               Bitcoin     BTC -0.007846  ...   
...                          ...                   ...     ...       ...  ...   
7046    2020-12-23  1.199000e+07            Aavegotchi    GHST  0.026175  ...   
7048    2020-12-23  9.224765e+06          Wing Finance    WING -0.209961  ...   
7083    2020-12-23  8.518168e+08               Uniswap     UNI -0.118426  ...   
7096    2020-12-23  8.345215e+07         Bridge Oracle     BRG -0.021064  ...   
7133    2020-12-23  3.322867e+07  Ducato Finance Token  DUCATO  0.023323  ...   

                      pVaR_5  VaR5%_group  port_weight  port_wret  port_ret  \
id_name time                                                                  
1       2016-01-08  0.043284            1     0.999054   0.042546  0.042515   
        2016-01-15  0.043284            1     0.998998  -0.218118 -0.217417   
        2016-01-22  0.054296            1     0.971184   0.047243  0.046603   
        2016-01-29  0.066992            2     0.967797  -0.007667  0.008692   
        2016-02-05  0.066992            2     0.998520   0.018445  0.018437   
...                      ...          ...          ...        ...       ...   
7046    2020-12-23  0.072897            3     0.000659  -0.000073 -0.047896   
7048    2020-12-23  0.190958           10     0.007676  -0.000824 -0.057792   
7083    2020-12-23  0.129703            8     0.191573  -0.012019 -0.199643   
7096    2020-12-23  0.317586           10     0.069440   0.007813 -0.057792   
7133    2020-12-23  0.157862            9     0.016793   0.007598 -0.015440   

                    port_size  port_e_wret  port_e_ret  port_excess_ret  \
id_name time                                                              
1       2016-01-08          2     0.021293    0.005066         0.042477   
        2016-01-15          2    -0.109168    0.240696        -0.217453   
        2016-01-22          2     0.024322    0.013208         0.046553   
        2016-01-29          2    -0.003961    0.250020         0.008649   
        2016-02-05          2     0.009236    0.006590         0.018393   
...                       ...          ...         ...              ...   
7046    2020-12-23         75    -0.001472   -0.113656        -0.047910   
7048    2020-12-23         76    -0.001412   -0.020518        -0.057805   
7083    2020-12-23         75    -0.000837   -0.128512        -0.199656   
7096    2020-12-23         76     0.001480   -0.020518        -0.057805   
7133    2020-12-23       

檢查加權是否一致

In [18]:
d1.groupby(['date', 'VaR5%_group'])['port_weight'].sum()#check


date        VaR5%_group
2016-01-08  1              1.0
            2              1.0
            3              1.0
            4              1.0
            5              1.0
                          ... 
2020-12-23  6              1.0
            7              1.0
            8              1.0
            9              1.0
            10             1.0
Name: port_weight, Length: 2590, dtype: float64

### TABLE1

In [19]:
#TABLE1
TABLE1_data = d1[['id','VaR_5','date','Beta','Size','Mom','Vhigh','Vlow','Abvol','Ivol','Vol','Illiq','Coskew','Max','Prc','Maxdprc','Prcvol','Stdprcvol']]
TABLE1_data = TABLE1_data[['VaR_5','Beta','Size','Mom','Vhigh','Vlow','Abvol','Ivol','Vol','Illiq','Coskew','Max','Prc','Maxdprc','Prcvol','Stdprcvol']].transform(lambda x: np.maximum(x.quantile(.01), np.minimum(x, x.quantile(.99))))
TABLE1 = TABLE1_data[['VaR_5','Beta','Size','Mom','Vhigh','Vlow','Abvol','Ivol','Vol','Illiq','Coskew','Max','Prc','Maxdprc','Prcvol','Stdprcvol']].describe(percentiles=[.05,.25,.5,.75,.95])
TABLE1 = TABLE1.drop(['count','min','max']).transpose()
TABLE1

,mean,std,5%,25%,50%,75%,95%
VaR_5,0.128007,0.071592,5.086152e-02,8.477493e-02,1.142617e-01,0.150690,0.251776
Beta,0.944745,0.396852,1.700824e-01,7.267737e-01,9.730508e-01,1.198148,1.552077
Size,16.728140,1.699641,1.470250e+01,1.542722e+01,1.634427e+01,17.602655,20.238055
Mom,-0.002673,0.209416,-3.415205e-01,-1.089629e-01,-6.692329e-03,0.090937,0.358092
Vhigh,0.166355,0.372401,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.000000
Vlow,0.202295,0.401713,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.000000
Abvol,3.473211,4.791200,-4.018714e+00,3.100237e-01,3.102521e+00,6.304738,12.107579
Ivol,0.008962,0.016739,6.125622e-04,1.925922e-03,3.846092e-03,0.008579,0.032064
Vol,0.090916,0.055539,3.685049e-02,5.752632e-02,7.718317e-02,0.106606,0.193196
Illiq,0.001558,0.009762,6.229631e-11,3.319708e-08,9.545904e-07,0.000019,0.001930


In [20]:
TABLE1.to_csv('TABLE1.csv')

### TABLE3

定義一個標示顯著值的函式

In [21]:
def p_star(df,df2,df_col):
    for i in df_col:
        df[i] = np.where(df2[i]<=0.01,df[i].astype(str)+'***',(np.where((df2[i]<=0.05)&(df2[i]>0.01),df[i].astype(str)+'**',(np.where((df2[i]>0.05)&(df2[i]<=0.1),df[i].astype(str)+'*',df[i])))))
    return df

In [22]:
import statsmodels.api as sm
from scipy import stats

TABLE3_data = d1.groupby(['date','VaR5%_group'])[['port_excess_ret','port_e_excess_ret']].apply(lambda x : x.drop_duplicates())
TABLE3_data = TABLE3_data.reset_index()
three_factor = pd.read_csv("C:/Users/USER/Desktop/MLY_RA/data/three_factor.csv")#引入三因子
model_data = pd.merge(TABLE3_data,three_factor,on ='date' ,how ='left')
model_data = model_data.dropna(subset=['CMOM'])

TABLE3_reg = model_data.groupby('VaR5%_group').apply(lambda x: sm.OLS(x['port_excess_ret'],sm.add_constant(x[['CMKT','CSMB','CMOM']])).fit(cov_type='HAC',cov_kwds={'maxlags' : 6},use_t=True).params).round(3)
TABLE3_reg_p = model_data.groupby('VaR5%_group').apply(lambda x: sm.OLS(x['port_excess_ret'],sm.add_constant(x[['CMKT','CSMB','CMOM']])).fit(cov_type='HAC',cov_kwds={'maxlags' : 6},use_t=True).pvalues)
TABLE3_reg = p_star(TABLE3_reg,TABLE3_reg_p,['const','CMKT','CSMB','CMOM']).rename(columns={'const':'equal_alpha'})
TABLE3_reg1 = model_data.groupby('VaR5%_group').apply(lambda x: sm.OLS(x['port_e_excess_ret'],sm.add_constant(x[['CMKT','CSMB','CMOM']])).fit(cov_type='HAC',cov_kwds={'maxlags' : 6},use_t=True).params).round(3)
TABLE3_reg1_p = model_data.groupby('VaR5%_group').apply(lambda x: sm.OLS(x['port_e_excess_ret'],sm.add_constant(x[['CMKT','CSMB','CMOM']])).fit(cov_type='HAC',cov_kwds={'maxlags' : 6},use_t=True).pvalues)
TABLE3_reg1 = p_star(TABLE3_reg1,TABLE3_reg1_p,['const','CMKT','CSMB','CMOM']).rename(columns={'const':'value_alpha'})
TABLE3_ = TABLE3_data.groupby('VaR5%_group')[['port_excess_ret','port_e_excess_ret']].mean()
TABLE3_ = TABLE3_.rename(columns={'port_excess_ret':'value-weighted-Excess-returns','port_e_excess_ret':'equal-weighted-Excess-returns'}).round(3)
TABLE3_P = TABLE3_data.groupby('VaR5%_group').apply(lambda x :stats.ttest_1samp(x[['port_excess_ret','port_e_excess_ret']],0).pvalue ).reset_index(name = 'P')
TABLE3_P[['value-weighted-Excess-returns','equal-weighted-Excess-returns']] = TABLE3_P['P'].apply(pd.Series)
TABLE3_P = TABLE3_P.set_index(['VaR5%_group'])
TABLE3_ = p_star(TABLE3_,TABLE3_P,['value-weighted-Excess-returns','equal-weighted-Excess-returns'])
TABLE3 = pd.concat([TABLE3_,TABLE3_reg['equal_alpha'],TABLE3_reg1['value_alpha']],axis=1)
cols = TABLE3.columns.to_list()
cols = [cols[0]]+[cols[3]]+[cols[1]]+[cols[2]]
TABLE3  =TABLE3[cols]
#TABLE3.loc['High-Low'] = TABLE3.loc['10']-TABLE3.loc['1']
TABLE3



,value-weighted-Excess-returns,value_alpha,equal-weighted-Excess-returns,equal_alpha
VaR5%_group,,,,
1,0.027***,-0.011**,0.014*,0.008**
2,0.003,-0.026***,0.005,-0.019***
3,0.023**,-0.029***,0.009,-0.01
4,0.027**,-0.031***,0.012,-0.018**
5,0.022*,-0.035***,0.006,-0.016**
6,0.028**,-0.032***,0.005,-0.028***
7,0.016,-0.045***,-0.006,-0.021***
8,0.025**,-0.035***,0.009,-0.024***
9,0.015,-0.034***,0.008,-0.04**


In [23]:
TABLE3.to_csv('TABLE3.csv')

### TABLE5

In [24]:
VARS = ['VaR_5','Beta','Size','Mom','Vhigh','Vlow','Abvol','Ivol','Vol','Illiq','Coskew','Max','Prc','Maxdprc','Prcvol','Stdprcvol']
VARSS = ['VaR5%_group']+VARS
TABLE5_data = d1[VARSS]
TABLE5 = TABLE5_data.groupby('VaR5%_group').mean().round(3)
TABLE5_P = TABLE5_data.groupby('VaR5%_group').apply(lambda x :stats.ttest_1samp(x[VARS].dropna(),0).pvalue ).reset_index(name = 'P')
TABLE5_P[VARS] = TABLE5_P['P'].apply(pd.Series)
TABLE5 = p_star(TABLE5,TABLE5_P,VARS)
TABLE5

,VaR_5,Beta,Size,Mom,Vhigh,Vlow,Abvol,Ivol,Vol,Illiq,Coskew,Max,Prc,Maxdprc,Prcvol,Stdprcvol
VaR5%_group,,,,,,,,,,,,,,,,
1,0.06***,0.67***,18.127***,0.008***,0.203***,0.183***,7.388***,0.002***,0.046***,0.0***,-0.652***,0.063***,-0.326***,-0.26***,14.574***,13.371***
2,0.085***,0.907***,17.614***,0.002,0.186***,0.198***,5.468***,0.003***,0.061***,0.0***,-1.797***,0.083***,-1.413***,-1.328***,12.792***,11.754***
3,0.094***,0.939***,17.195***,0.002,0.164***,0.206***,4.597***,0.003***,0.067***,0.0***,-2.231***,0.091***,-1.906***,-1.814***,11.94***,10.979***
4,0.102***,0.956***,16.943***,-0.001,0.173***,0.209***,4.051***,0.004***,0.072***,0.0***,-2.525***,0.097***,-2.202***,-2.105***,11.395***,10.477***
5,0.11***,0.978***,16.707***,-0.005***,0.152***,0.212***,3.613***,0.005***,0.077***,0.0***,-2.518***,0.103***,-2.413***,-2.309***,10.911***,10.016***
6,0.118***,0.991***,16.547***,-0.004**,0.158***,0.206***,3.228***,0.006***,0.083***,0.002***,-2.537***,0.109***,-2.538***,-2.433***,10.575***,9.698***
7,0.128***,1.012***,16.388***,-0.007***,0.155***,0.212***,2.845***,0.007***,0.092***,0.006,-2.671***,0.118***,-2.755***,-2.64***,10.13***,9.251***
8,0.142***,1.022***,16.208***,-0.008***,0.153***,0.208***,2.402***,0.009***,0.101***,0.002***,-2.256***,0.131***,-2.845***,-2.723***,9.691***,8.796***
9,0.167***,1.026***,15.91***,-0.009***,0.151***,0.205***,1.333***,0.013***,0.119***,0.007***,-1.976***,0.154***,-3.202***,-3.066***,8.619***,7.767***


In [25]:
TABLE5.to_csv('TABLE5.csv')

### TABLE7

In [60]:
TABLE7_data = d1[['id','date','week_ret','VaR_5','Beta','Size','Mom','Vhigh','Vlow','Abvol','Ivol','Vol','Illiq','Coskew','Max','Prc','Maxdprc','Prcvol','Stdprcvol']]
TABLE7_data['date'] = pd.to_datetime(TABLE7_data['date'])
TABLE7_data = TABLE7_data.set_index(['id','date'])
#TABLE7_data[VARS] = TABLE7_data.groupby('id')[VARS].shift(1)#此段程式碼有待測試(FAMAMACBETH會報錯))

C:\Users\USER\AppData\Local\Temp\ipykernel_22048\2241898249.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TABLE7_data['date'] = pd.to_datetime(TABLE7_data['date'])


In [27]:
TABLE7_data.dtypes

week_ret     float64
VaR_5        float64
Beta         float64
Size         float64
Mom          float64
Vhigh          int32
Vlow           int32
Abvol        float64
Ivol         float64
Vol          float64
Illiq        float64
Coskew       float64
Max          float64
Prc          float64
Maxdprc      float64
Prcvol       float64
Stdprcvol    float64
dtype: object

In [61]:
import math
import numpy as np
import pandas as pd
import statsmodels.api as sm

def FamaMacBeth_summary(DF,
                        reg_total,
                        reg_order,
                        reg_names=None,
                        params_format='{:.3f}',
                        tvalues_format='{:.2f}'):

    '''
    A function for Fama-MacBeth regression and results summary.

    Parameters
    ----------
    DF: DataFrame
        A panel date of which multi-index is stock and month (datetime64[ns]),
        containing all the dependent and independent variables.
    reg_total: list
        A list containing all of dependent variable and independent
        variables, e.g., ['Y', 'X1', ...].The function automatically will calculate result of the following list:[['Y', 'X1']、['Y', 'X1','X2' ...]...['Y', 'X1', ...]].
    reg_order: list
        The order of independent variables in result table.
    reg_names: list
        The names for each regression.
    params_format: str
        The number of decimal places for parameters, e.g., '{:.3f}'.
    tvalues_format: str
        The number of decimal places for t-values, e.g., '{:.2f}'.
    '''

    # Create a DataFrame
    reg_lst = []
    for i in reversed(range(2,len(reg_total)+1)):
        for l in [reg_total[:i]]:
            reg_lst+=[l]
    rows = sum([[var, f'{var}_t'] for var in ['const'] + reg_order], [])
    if reg_names is None:
        reg_names = [f'({i+1})' for i in range(len(reg_lst))]
    show = pd.DataFrame(index=rows, columns=reg_names)

    for reg, reg_name in zip(reg_lst, reg_names):
        df = DF.loc[:, reg].copy().dropna()
        T = len(df.index.get_level_values(df.index.names[1]).unique())
        lag = math.floor(4*(T/100)**(2/9))
        fmb = FamaMacBeth(df[reg[0]], sm.add_constant(df[reg[1:]]))
        # Newey-West adjust
        fmb = fmb.fit(cov_type='kernel', bandwidth=lag)

        # params, tvalues(tstats) and pvalues
        params = fmb.params
        tvalues = fmb.tstats
        pvalues = fmb.pvalues

        # Obs.
        total_obs = fmb.nobs
        # mean_obs = fmb.time_info['mean']

        # average rsquared_adj
        dft = df.reset_index(level=df.index.names[0], drop=True).copy()
        rsquared_adj = []
        for time in dft.index.unique():
            dftm = dft.loc[time].copy()
            ols = sm.OLS(dftm[reg[0]], sm.add_constant(dftm[reg[1:]])).fit()
            rsquared_adj.append(ols.rsquared_adj)
        ar2a = np.mean(rsquared_adj)

        # params and significance
        ps_lst = []
        for param, pvalue in zip(params, pvalues):
            param = params_format.format(param)
            if (pvalue <= 0.1) & (pvalue > 0.05):
                param = param + '*'
            elif (pvalue <= 0.05) & (pvalue > 0.01):
                param = param + '**'
            elif pvalue <= 0.01:
                param = param + '***'
            ps_lst.append(param)

        # params and tvalues
        tvalues = [tvalues_format.format(t) for t in tvalues]
        t_lst = [f'({t})' for t in tvalues]
        pt_lst = [[i, j] for i, j in zip(ps_lst, t_lst)]

        # put them in place
        for var, pt in zip(['const'] + reg[1:], pt_lst):
            show.loc[var, reg_name] = pt[0]
            show.loc[f'{var}_t', reg_name] = pt[1]
        show.loc['No. Obs.', reg_name] = str(total_obs)
        show.loc['Adj. R²', reg_name] = '{:.2f}%'.format(ar2a * 100)
    
    row = sum([[var, ''] for var in reg_total[1:]], [])
        
    rename_index = sum([[var, ''] for var in ['Intercept'] + reg_order], [])
    show.index = rename_index + row +['No. Obs.', 'Adj. R²']

    return show.dropna(axis=0, how='all').fillna('')
#計算因子負荷有兩種方法：

#1.使用股票和因子報酬率在時序上迴歸得到的迴歸係數作為因子負荷（例如，使用個股報酬率和HML 因子報酬率的時序迴歸係數作為個股在HML 上的因子負荷）；

#2.像Barra 那樣直接使用firm characteristics 作為因子負荷（例如，直接用個股的P/B 取值經過必要的標準化後作為因子負荷）。此處應該是這種


TABLE7 = FamaMacBeth_summary(TABLE7_data,reg_total=['week_ret','VaR_5','Beta','Size','Mom','Vhigh','Vlow','Abvol','Ivol','Vol','Illiq','Coskew','Max','Prc','Maxdprc','Prcvol','Stdprcvol'],reg_order=[50]).round(3)
clos = TABLE7.shape[0]
for i in range(clos):
    TABLE7.iloc[i]  = TABLE7.iloc[i][::-1]
    
TABLE7

C:\Users\USER\AppData\Local\Temp\ipykernel_22048\3760048837.py:48: MissingValueWarning: 10 of the time-series regressions cannot be estimated due to deficient rank.
  fmb = FamaMacBeth(df[reg[0]], sm.add_constant(df[reg[1:]]))
C:\Users\USER\AppData\Local\Temp\ipykernel_22048\3760048837.py:48: MissingValueWarning: 10 of the time-series regressions cannot be estimated due to deficient rank.
  fmb = FamaMacBeth(df[reg[0]], sm.add_constant(df[reg[1:]]))
C:\Users\USER\AppData\Local\Temp\ipykernel_22048\3760048837.py:48: MissingValueWarning: 10 of the time-series regressions cannot be estimated due to deficient rank.
  fmb = FamaMacBeth(df[reg[0]], sm.add_constant(df[reg[1:]]))
C:\Users\USER\AppData\Local\Temp\ipykernel_22048\3760048837.py:48: MissingValueWarning: 10 of the time-series regressions cannot be estimated due to deficient rank.
  fmb = FamaMacBeth(df[reg[0]], sm.add_constant(df[reg[1:]]))
C:\Users\USER\AppData\Local\Temp\ipykernel_22048\3760048837.py:48: MissingValueWarning: 10 o

,(1),(2),(3),(4),(5),(6),(7),(8),(9),(10),(11),(12),(13),(14),(15),(16)
Intercept,-0.008,-0.003,-0.126***,-0.131***,-0.142***,-0.124***,-0.270***,-0.216***,-0.263***,-0.274***,-0.270***,-0.231***,-0.292***,-0.296***,-0.397***,-0.391***
,(-0.76),(-0.31),(-5.92),(-5.97),(-6.31),(-5.22),(-8.00),(-6.81),(-8.27),(-8.54),(-8.86),(-9.28),(-9.61),(-9.38),(-11.12),(-10.15)
VaR_5,0.147***,0.112**,0.181***,0.174***,0.192***,0.146***,0.045,-0.313***,-0.547***,-0.532***,-0.495***,-0.614***,-0.655***,-0.645***,-0.425***,-0.406***
,(3.04),(2.23),(3.40),(3.40),(3.81),(2.82),(0.89),(-3.80),(-4.80),(-4.77),(-4.87),(-7.19),(-7.64),(-6.71),(-4.01),(-3.88)
Beta,,-0.001,-0.005,-0.005,-0.003,-0.001,-0.003,0.007,-0.003,-0.004,-0.006,-0.001,0.003,0.002,0.002,0.000
,,(-0.18),(-0.62),(-0.55),(-0.42),(-0.13),(-0.42),(1.01),(-0.39),(-0.58),(-0.82),(-0.13),(0.44),(0.29),(0.19),(0.04)
Size,,,0.007***,0.007***,0.007***,0.006***,0.018***,0.016***,0.016***,0.017***,0.017***,0.014***,0.025***,0.025***,0.015***,0.015***
,,,(6.62),(6.61),(6.66),(5.87),(7.67),(6.86),(7.37),(7.67),(7.74),(8.08),(9.09),(9.33),(7.59),(8.10)
Mom,,,,-0.077***,-0.081***,-0.090***,-0.092***,-0.078***,-0.092***,-0.089***,-0.086***,-0.161***,-0.173***,-0.167***,-0.204***,-0.207***
,,,,(-3.51),(-3.99),(-4.35),(-4.39),(-6.44),(-7.85),(-7.70),(-7.42),(-16.35),(-17.26),(-14.56),(-21.28),(-17.11)


In [29]:
TABLE7.to_csv('TABLE7.csv')

In [30]:
j = ['week_ret','VaR_5','Beta','Size','Mom','Vhigh','Vlow','Abvol','Ivol','Vol','Illiq','Coskew','Max','Prc','Maxdprc','Prcvol','Stdprcvol']
sums = []
for i in reversed(range(2,len(j)+1)):
    for l in [j[:i]]:
        sums+=[l]
sums

[['week_ret',
  'VaR_5',
  'Beta',
  'Size',
  'Mom',
  'Vhigh',
  'Vlow',
  'Abvol',
  'Ivol',
  'Vol',
  'Illiq',
  'Coskew',
  'Max',
  'Prc',
  'Maxdprc',
  'Prcvol',
  'Stdprcvol'],
 ['week_ret',
  'VaR_5',
  'Beta',
  'Size',
  'Mom',
  'Vhigh',
  'Vlow',
  'Abvol',
  'Ivol',
  'Vol',
  'Illiq',
  'Coskew',
  'Max',
  'Prc',
  'Maxdprc',
  'Prcvol'],
 ['week_ret',
  'VaR_5',
  'Beta',
  'Size',
  'Mom',
  'Vhigh',
  'Vlow',
  'Abvol',
  'Ivol',
  'Vol',
  'Illiq',
  'Coskew',
  'Max',
  'Prc',
  'Maxdprc'],
 ['week_ret',
  'VaR_5',
  'Beta',
  'Size',
  'Mom',
  'Vhigh',
  'Vlow',
  'Abvol',
  'Ivol',
  'Vol',
  'Illiq',
  'Coskew',
  'Max',
  'Prc'],
 ['week_ret',
  'VaR_5',
  'Beta',
  'Size',
  'Mom',
  'Vhigh',
  'Vlow',
  'Abvol',
  'Ivol',
  'Vol',
  'Illiq',
  'Coskew',
  'Max'],
 ['week_ret',
  'VaR_5',
  'Beta',
  'Size',
  'Mom',
  'Vhigh',
  'Vlow',
  'Abvol',
  'Ivol',
  'Vol',
  'Illiq',
  'Coskew'],
 ['week_ret',
  'VaR_5',
  'Beta',
  'Size',
  'Mom',
  'Vhigh',
  

In [62]:
TABLE7_data[VARS] = TABLE7_data.groupby('id')[VARS].shift(1)#此段程式碼有待測試(FAMAMACBETH會報錯))，原因不明，只知道有時能成功有時會失敗
#VARSSS =(pd.Series(VARS)+"P").tolist()
#TABLE7_data[VARSSS] = TABLE7_data.groupby('id')[VARS].shift(1)

In [63]:
import math
import numpy as np
import pandas as pd
import statsmodels.api as sm

def FamaMacBeth_summary(DF,
                        reg_total,
                        reg_order,
                        reg_names=None,
                        params_format='{:.3f}',
                        tvalues_format='{:.2f}'):

    '''
    A function for Fama-MacBeth regression and results summary.

    Parameters
    ----------
    DF: DataFrame
        A panel date of which multi-index is stock and month (datetime64[ns]),
        containing all the dependent and independent variables.
    reg_total: list
        A list containing all of dependent variable and independent
        variables, e.g., ['Y', 'X1', ...].The function automatically will calculate result of the following list:[['Y', 'X1']、['Y', 'X1','X2' ...]...['Y', 'X1', ...]].
    reg_order: list
        The order of independent variables in result table.
    reg_names: list
        The names for each regression.
    params_format: str
        The number of decimal places for parameters, e.g., '{:.3f}'.
    tvalues_format: str
        The number of decimal places for t-values, e.g., '{:.2f}'.
    '''

    # Create a DataFrame
    reg_lst = []
    for i in reversed(range(2,len(reg_total)+1)):
        for l in [reg_total[:i]]:
            reg_lst+=[l]
    rows = sum([[var, f'{var}_t'] for var in ['const'] + reg_order], [])
    if reg_names is None:
        reg_names = [f'({i+1})' for i in range(len(reg_lst))]
    show = pd.DataFrame(index=rows, columns=reg_names)

    for reg, reg_name in zip(reg_lst, reg_names):
        df = DF.loc[:, reg].copy().dropna()
        T = len(df.index.get_level_values(df.index.names[1]).unique())
        lag = math.floor(4*(T/100)**(2/9))
        fmb = FamaMacBeth(df[reg[0]], sm.add_constant(df[reg[1:]]))
        # Newey-West adjust
        fmb = fmb.fit(cov_type='kernel', bandwidth=lag)

        # params, tvalues(tstats) and pvalues
        params = fmb.params
        tvalues = fmb.tstats
        pvalues = fmb.pvalues

        # Obs.
        total_obs = fmb.nobs
        # mean_obs = fmb.time_info['mean']

        # average rsquared_adj
        dft = df.reset_index(level=df.index.names[0], drop=True).copy()
        rsquared_adj = []
        for time in dft.index.unique():
            dftm = dft.loc[time].copy()
            ols = sm.OLS(dftm[reg[0]], sm.add_constant(dftm[reg[1:]])).fit()
            rsquared_adj.append(ols.rsquared_adj)
        ar2a = np.mean(rsquared_adj)

        # params and significance
        ps_lst = []
        for param, pvalue in zip(params, pvalues):
            param = params_format.format(param)
            if (pvalue <= 0.1) & (pvalue > 0.05):
                param = param + '*'
            elif (pvalue <= 0.05) & (pvalue > 0.01):
                param = param + '**'
            elif pvalue <= 0.01:
                param = param + '***'
            ps_lst.append(param)

        # params and tvalues
        tvalues = [tvalues_format.format(t) for t in tvalues]
        t_lst = [f'({t})' for t in tvalues]
        pt_lst = [[i, j] for i, j in zip(ps_lst, t_lst)]

        # put them in place
        for var, pt in zip(['const'] + reg[1:], pt_lst):
            show.loc[var, reg_name] = pt[0]
            show.loc[f'{var}_t', reg_name] = pt[1]
        show.loc['No. Obs.', reg_name] = str(total_obs)
        show.loc['Adj. R²', reg_name] = '{:.2f}%'.format(ar2a * 100)
    
    row = sum([[var, ''] for var in reg_total[1:]], [])
        
    rename_index = sum([[var, ''] for var in ['Intercept'] + reg_order], [])
    show.index = rename_index + row +['No. Obs.', 'Adj. R²']

    return show.dropna(axis=0, how='all').fillna('')
#計算因子負荷有兩種方法：

#1.使用股票和因子報酬率在時序上迴歸得到的迴歸係數作為因子負荷（例如，使用個股報酬率和HML 因子報酬率的時序迴歸係數作為個股在HML 上的因子負荷）；

#2.像Barra 那樣直接使用firm characteristics 作為因子負荷（例如，直接用個股的P/B 取值經過必要的標準化後作為因子負荷）。此處應該是這種


TABLE7 = FamaMacBeth_summary(TABLE7_data,reg_total=['week_ret']+['VaR_5','Beta','Size','Mom','Vhigh','Vlow','Abvol','Ivol','Vol','Illiq','Coskew','Max','Prc','Maxdprc','Prcvol','Stdprcvol'],reg_order=[50]).round(3)
clos = TABLE7.shape[0]
for i in range(clos):
    TABLE7.iloc[i]  = TABLE7.iloc[i][::-1]
    
TABLE7

C:\Users\USER\AppData\Local\Temp\ipykernel_22048\2379427116.py:48: MissingValueWarning: 22 of the time-series regressions cannot be estimated due to deficient rank.
  fmb = FamaMacBeth(df[reg[0]], sm.add_constant(df[reg[1:]]))
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1794: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1794: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1794: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1794: RuntimeWarning: invalid value encountered in scala

TypeError: object of type 'numpy.float64' has no len()

In [ ]:
'''
d = data1.copy().reset_index()
d = d.set_index('time')
d = ret_for_prior_week(d,7)#以處所計算的week_ret、port_ret、port_e_ret都是前一個禮拜的
d1 = d['2016/01/01':'2020/12/31']
d1 = d1.groupby(['id','year','week']).first().reset_index()
dd1 = d1.reset_index(drop = True)
d1['time'] = pd.to_datetime(d1['date'])
d1 = d1.set_index(['id_name','time'],drop = True)
d1 = winsorwize(d1)
d1['pVaR_5'] = d1.groupby('id')['VaR_5'].shift(1)
d1 = d1.dropna(subset=['pVaR_5'])
d1['VaR5%_group'] = d1.groupby(['date'])['pVaR_5'].transform( lambda x: pd.qcut(x, q=10, labels=range(1,11)))
port_cap = d1.groupby(['VaR5%_group','date'])['market_cap'].transform('sum')
d1['port_weight'] = d1['market_cap']/port_cap
d1['port_wret'] = d1['week_ret']*d1['port_weight']
d1['port_ret'] = d1.groupby(['VaR5%_group','date'])['port_wret'].transform('sum')#計算投組的市值加權報酬

port_size = d1.groupby(['VaR5%_group','date']).size().reset_index(name='port_size')
d1 = pd.merge(d1,port_size,how = 'left',on = ['VaR5%_group','date'])
d1 = d1.reset_index(drop = True)
d1['time'] = pd.to_datetime(d1['date'])
d1['id_name'] = d1['id']
d1 = d1.set_index(['id_name','date'],drop=True)
d1
'''

d:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


id  year  week       close    volume_24h    market_cap  \
id_name date                                                                   
1       2016-01-08     1  2016     2  453.230011  5.699300e+07  6.825700e+09   
        2016-01-15     1  2016     3  364.330994  1.533510e+08  5.496598e+09   
        2016-01-22     1  2016     4  382.492004  9.154660e+07  5.780764e+09   
        2016-01-29     1  2016     5  379.473999  8.612530e+07  5.745986e+09   
        2016-02-05     1  2016     6  386.549011  4.382500e+07  5.865137e+09   
...                  ...   ...   ...         ...           ...           ...   
7046    2020-12-23  7046  2020    52    0.612376  2.409312e+07  1.199000e+07   
7048    2020-12-23  7048  2020    52   14.189963  4.047784e+06  9.224765e+06   
7083    2020-12-23  7083  2020    52    3.313970  8.383838e+08  8.518168e+08   
7096    2020-12-23  7096  2020    52    0.010429  2.684505e+06  8.345215e+07   
7133    2020-12-23  7133  2020    52   26.164306  4.290594e+05  3.322867e+07   

                                    name  symbol       ret     VaR_5  ...  \
id_name date                                                          ...   
1       2016-01-08               Bitcoin     BTC -0.010574  0.043284  ...   
        2016-01-15               Bitcoin     BTC -0.166434  0.054296  ...   
        2016-01-22               Bitcoin     BTC -0.070088  0.066992  ...   
        2016-01-29               Bitcoin     BTC -0.002145  0.066992  ...   
        2016-02-05               Bitcoin     BTC -0.007846  0.058080  ...   
...                                  ...     ...       ...       ...  ...   
7046    2020-12-23            Aavegotchi    GHST  0.026175  0.069015  ...   
7048    2020-12-23          Wing Finance    WING -0.209961  0.150436  ...   
7083    2020-12-23               Uniswap     UNI -0.118426  0.118214  ...   
7096    2020-12-23         Bridge Oracle     BRG -0.021064  0.317586  ...   
7133    2020-12-23  Ducato Finance Token  DUCATO  0.023323  0.157862  ...   

                    sq_MKT_excess    Coskew  week_ret    pVaR_5  VaR5%_group  \
id_name date                                                                   
1       2016-01-08   9.234234e-05  0.023183  0.042586  0.043284            1   
        2016-01-15   2.521600e-02 -0.017512 -0.218337  0.043284            1   
        2016-01-22   4.316017e-03 -0.012135  0.048645  0.054296            1   
        2016-01-29   1.283727e-07  0.024383 -0.007922  0.066992            2   
        2016-02-05   2.132850e-05 -0.017336  0.018473  0.066992            2   
...                           ...       ...       ...       ...          ...   
7046    2020-12-23   9.491598e-04       NaN -0.110384  0.072897            3   
7048    2020-12-23   9.491598e-04       NaN -0.107319  0.190958           10   
7083    2020-12-23   9.491598e-04       NaN -0.062740  0.129703            8   
7096    2020-12-23   9.491598e-04       NaN  0.112510  0.317586           10   
7133    2020-12-23   9.491598e-04       NaN  0.452439  0.157862            9   

                    port_weight  port_wret  port_ret  port_size       time  
id_name date                                                                
1       2016-01-08     0.999054   0.042546  0.042515          2 2016-01-08  
        2016-01-15     0.998998  -0.218118 -0.217417          2 2016-01-15  
        2016-01-22     0.971184   0.047243  0.046603          2 2016-01-22  
        2016-01-29     0.967797  -0.007667  0.008692          2 2016-01-29  
        2016-02-05     0.998520   0.018445  0.018437          2 2016-02-05  
...                         ...        ...       ...        ...        ...  
7046    2020-12-23     0.000659  -0.000073 -0.047896         75 2020-12-23  
7048    2020-12-23     0.007676  -0.000824 -0.057792         76 2020-12-23  
7083    2020-12-23     0.191573  -0.012019 -0.199643         75 2020-12-23  
7096    2020-12-23     0.069440   0.007813 -0.057792         76 2020-12-23  
7133    2020-1

In [ ]:
df = pd.DataFrame(data={'a':[np.nan,2,3,4,5],"b":['a','b','c','d','e'],'k':[15,2,3,74,5],"e":[1,1,1,2,2]}).set_index('b')
df2 = pd.DataFrame(data = {'a':[.1,.2,.3,.4,.5],"b":['a','b','c','d','e'],'k':[.01,2,.03,.14,5]}).set_index('b')
#df = np.where(df2<=0.01,df.astype(str)+'***',(np.where((df2<=0.05)&(df2>0.01),df.astype(str)+'**',(np.where((df2>0.05)&(df2<=0.1),df.astype(str)+'*',df)))))
#p_star(df,df2)

5

In [ ]:
l = winsorwize_nonret2(df,['a','k'],['e'])
l

,a,k,e
b,,,
a,NaN,3,1
b,3.0,3,1
c,3.0,3,1
d,5.0,74,2
e,5.0,74,2


In [ ]:
def winsorwize_nonret2(df,df_list,group_list):
    for i in df_list:
        df[i] = df.groupby(group_list)[i].transform(lambda x: winsorize(x,limits=[0.01,0.01],nan_policy='omit'))
    return df

In [ ]:
mask = df['a'].notna()
a = df.loc[mask,'a'] = winsorize(df['a'].loc[mask],limits = [0.4,0.4])
a

masked_array(data=[3., 3., 4., 4.],
             mask=False,
       fill_value=1e+20)